In [0]:
def reduce_memory_usage(df, verbose=True):
  numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
  start_mem = df.memory_usage().sum() / 1024 ** 2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == "int":
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if (
                  c_min > np.finfo(np.float16).min
                  and c_max < np.finfo(np.float16).max
              ):
                  df[col] = df[col].astype(np.float16)
              elif (
                  c_min > np.finfo(np.float32).min
                  and c_max < np.finfo(np.float32).max
              ):
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024 ** 2
  if verbose:
      print(
          "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
              end_mem, 100 * (start_mem - end_mem) / start_mem
          )
      )
  return df

In [0]:
#---------------------------Período Desenv-------------------------------
def decil_model(base, var_chave , bins, var_anomes ,safra_desenv, score):

  decil_desenv = base[base[str(var_anomes)] <= safra_desenv]

  #retorna os bins para os decis de desenvolvimento -> Todos, para calculo do PSI
  res, fx_desenv = pd.qcut(decil_desenv[str(score)], bins, retbins=True, duplicates='drop')

  #caso o modelo retorne algum valor que não existe, estou mudando manualmente os bins
  fx_desenv[0] = -10

  decil_desenv = decil_desenv.groupby(pd.qcut(decil_desenv[score], bins, duplicates='drop'))[str(var_chave)].count()

  #DataFrame com os decis do desenvolvimento 
  decil_desenv = pd.DataFrame(decil_desenv).reset_index().reset_index().rename(columns={'index':'gp', str(score):'fx_desenv'})

  decil_desenv['pct_desenv'] = decil_desenv[str(var_chave)]/sum(decil_desenv[str(var_chave)])

  #------------------------------Produção-------------------------------

  #Consulta
  decil_prod = base

  #Realiza as quebras no dataframe de produção
  decil_prod = decil_prod.merge(pd.DataFrame(pd.cut(decil_prod[str(score)],fx_desenv)).rename(columns={str(score):'fx_desenv'}),how='left', right_index=True, left_index=True)

  #Cria os grupos -> o Fillna 0 funciona pois para a primeira faixa a função cut realiza uma quebra menor no primeiro numero
  decil_prod = decil_prod.merge(decil_desenv[['fx_desenv','gp']],how='left',on='fx_desenv').fillna({"gp": 0})

  #Transformo a variavel fx_desenv em string para realizar a conversa em spark dataframe, posteriormente em uma view para criar os graficos stacked 100%
  decil_prod['fx_desenv'] = decil_prod['fx_desenv'].astype(str)

  return decil_prod

In [0]:
#decil -> base com os decil calculado pela função acima

def psi(decil, var_chave , var_anomes, anomes_desenv, fl_grafico):

  import seaborn as sns
  import matplotlib.pyplot as plt

  sns.set_style("white")
  sns.set_context("talk", font_scale=0.7)

  #DataFrame para guardar os resultados
  psi_total = None

  safras = decil[str(var_anomes)].unique()

  #distribuição dos decis nas safras de desenvolvimento
  gp_desenv = decil[decil[str(var_anomes)] <= anomes_desenv][[str(var_chave),'gp']].groupby(['gp']).count().rename(columns={str(var_chave):'qt'}).reset_index()

  gp_desenv['pct_desenv'] = gp_desenv['qt']/sum(gp_desenv['qt'])
  
  gp_desenv.drop('qt', axis=1, inplace=True)

  for safra in safras:

    #distribuição dos decis nas safras de desenvolvimento
    gp_safra = decil[decil[str(var_anomes)] == safra][[str(var_chave),'fx_desenv','gp']].groupby(['gp','fx_desenv']).count().rename(columns={str(var_chave):'qt'}).reset_index()

    #QUANTIDADE TOTAL
    psi_table = gp_safra.groupby(['gp','fx_desenv']).agg({'qt': 'sum'}).reset_index().rename(columns={"qt": "Total"})

    #PERCENTUAL TOTAL
    psi_table['pct_total'] = psi_table['Total']/sum(psi_table['Total'])

    #AGRUPANDO PERCENTUAL DO DECIL PERIODO DESENVOLVIMENTO
    psi_table = psi_table.merge(gp_desenv, how='left', on='gp')

    #PSI
    psi_table['PSI'] = (psi_table['pct_total'] - psi_table['pct_desenv']) * np.log(psi_table['pct_total'] / psi_table['pct_desenv'])

    #safra
    psi_table[str(var_anomes)] = safra

    #Appenda o resultado do processamento para uma tabela agrupada
    if psi_total is None:
      psi_total = psi_table
    else:
      psi_total = psi_total.append(psi_table)

  psi_total[str(var_anomes)] = psi_total[str(var_anomes)].astype(str)
  
  if fl_grafico == 0:
    return psi_total
  
  plot = psi_total.groupby(str(var_anomes)).agg(psi_tot=('PSI','sum')).reset_index().copy()

  plot['limit_bom'] = 0.1
  plot['limit_med'] = 0.2
  plot['limit_mau'] = 0.4
  f, ax = plt.subplots(1,figsize=(30,4))

  plt.fill_between(plot[str(var_anomes)], plot['limit_mau'], label='Atenção', color='red', alpha=0.1)
  plt.fill_between(plot[str(var_anomes)], plot['limit_med'], label='Médio', color='orange', alpha=0.5)
  plt.fill_between(plot[str(var_anomes)], plot['limit_bom'], label='Bom', color='green', alpha=0.6)

  plt.plot(plot[str(var_anomes)], plot['psi_tot'], marker='o',label='psi', color='blue', markersize=10)

  x = plot[str(var_anomes)]
  y = plot['psi_tot']
  n = plot['psi_tot']

  for i, v in enumerate(n):
    plt.text(x[i], y[i]+0.05, "{:.02}".format(v), fontsize=12, weight='bold', style='italic', bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 3})

  ax.set_ylim(bottom=0)
  #ax.set_ylim(top=0.5)
  plt.legend(loc='best', fontsize=10)
  plt.title('PSI Modelo', fontsize=18, weight='bold')
  sns.despine(bottom = True, left = True)
  plt.show()


In [0]:

def gains_table_tot(decil, var_chave, var_anomes, anomes_desenv, flg_bad):

  #contador de loops
  cont = 0

  #selecao das safras
  safras = pd.DataFrame(decil[str(var_anomes)].unique())

  #Adiciona o consolidado para processamento do KS historico
  safras.loc[len(safras)+1] = 'Consolidado'
  
  #distribuição dos decis nas safras de desenvolvimento
  gp_desenv = decil[decil[str(var_anomes)] <= anomes_desenv][[str(var_chave),'gp']].groupby(['gp']).count().rename(columns={str(var_chave):'qt'}).reset_index()

  gp_desenv['pct_desenv'] = gp_desenv['qt']/sum(gp_desenv['qt'])
  
  gp_desenv.drop('qt', axis=1, inplace=True)


  for safra in np.array(safras):

    if safra[0] == 'Consolidado':
      df_gains = decil[[str(var_anomes),'fx_desenv',str(flg_bad),str(var_chave),'gp']].groupby([str(var_anomes),'fx_desenv',str(flg_bad),'gp']).count().reset_index().rename(columns={str(var_chave):'QT'})
    else:
      df_gains = decil[decil[str(var_anomes)] == safra[0]][[str(var_anomes),'fx_desenv',str(flg_bad),str(var_chave),'gp']].groupby([str(var_anomes),'fx_desenv',str(flg_bad),'gp']).count().reset_index().rename(columns={str(var_chave):'QT'})

    df_gains.fillna(0,inplace=True)    

    #Bons
    gains_table = df_gains[df_gains[str(flg_bad)] == 0].groupby(['gp','fx_desenv']).agg({'QT': 'sum'}).reset_index().rename(columns={"QT": "qtd_bom"})

    #Maus
    gains_table = gains_table.merge(df_gains[df_gains[str(flg_bad)] == 1].groupby('gp').agg({'QT': 'sum'}).reset_index().rename(columns={"QT": "qtd_mau"}) , how='left', on='gp')

    #ALGUNS NÃO POSSUEM CONTRATOS NOS DECIS, PREENCHENDO COM ZEROS
    gains_table.fillna(0, inplace=True)

    #ACUM BONS E MAUS
    gains_table = gains_table.join(gains_table.cumsum().rename(columns={'qtd_bom': 'acum_bom', 'qtd_mau': 'acum_mau'}).drop(['gp','fx_desenv'],axis = 1))

    #TOTAL DE PROPOSTAS
    gains_table['total'] = gains_table['qtd_bom'] + gains_table['qtd_mau']

    #PERCENTUAL DO GRUPO EM RELACAO AO TOTAL
    #gains_table['pct_total'] = (gains_table['qtd_bom'] + gains_table['qtd_mau'])/sum((gains_table['qtd_bom'] + gains_table['qtd_mau']))

    gains_table = gains_table.merge(gp_desenv, how='left', on='gp')

    #BAD INTERVALAR
    gains_table['bad_interval'] = gains_table['qtd_mau'] / gains_table['total']

    #MAU INTERVALAR
    gains_table['mau_intervalar'] = gains_table['qtd_mau'] / sum(gains_table['qtd_mau'])

    #BOM INTERVALAR
    gains_table['bom_intervalar'] = gains_table['qtd_bom'] / sum(gains_table['qtd_bom'])

    #PCT ACUM MAUS
    gains_table['pct_acum_mau'] = gains_table['acum_mau'] / sum(gains_table['qtd_mau'])

    #PCT ACUM BONS
    gains_table['pct_acum_bons'] = gains_table['acum_bom'] / sum(gains_table['qtd_bom'])

    #LAG BONS CALCULO GINI
    calc_gini = gains_table[['gp','pct_acum_bons']].copy()
    calc_gini['gp'] = calc_gini['gp'] + 1
    calc_gini.rename(columns={'pct_acum_bons': 'lag_bons'}, inplace=True)

    gains_table = gains_table.merge(calc_gini, how='left',on='gp')

    #ALGUNS NÃO POSSUEM CONTRATOS NOS DECIS, PREENCHENDO COM ZEROS
    gains_table.fillna(0, inplace=True)

    #KS2
    gains_table['ks2'] = abs(gains_table['pct_acum_mau'] - gains_table['pct_acum_bons'])

    #gini
    gains_table['Gini'] = gains_table['mau_intervalar'] * (gains_table['pct_acum_bons']+gains_table['lag_bons']) * 0.5

    #safra
    gains_table['Safra'] = safra[0]

    if cont == 0:
      gains_total = gains_table
    else:
      gains_total = gains_total.append(gains_table)

    cont += 1

  return gains_total

In [0]:
def plot_ks(gains_total):
  
  import seaborn as sns
  
  plot_ks = gains_total.groupby(['Safra']).agg(ks2=('ks2','max')).reset_index().copy()
  plot_ks = plot_ks[plot_ks['ks2'] < 1].reset_index()
  plot_ks['Safra'] = plot_ks['Safra'].astype(str)
  
  f, ax = plt.subplots(1,figsize=(30,3))
  sns.lineplot(data=plot_ks, x='Safra', y='ks2', markers=True, marker='o',label='ks2', color='blue', markersize=12)
  ax.set_ylim(bottom=0)
  ax.set_ylim(top=1)
  plt.title('KS2', fontsize=20, weight='bold')

  x = plot_ks['Safra']
  y = plot_ks['ks2']
  n = plot_ks['ks2']

  for i, v in enumerate(n):
    plt.text(x[i], y[i]+0.05, "{:.2%}".format(v), fontsize=11, weight='bold', style='italic', bbox={'facecolor': 'gray', 'alpha': 0.3, 'pad': 5})

  sns.despine(bottom = True, left = True)
  plt.show()